In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
import pickle

In [4]:
# get data
data = pd.read_parquet("combined_air_travel_data.parquet")
weather_data = pd.read_parquet('weather.parquet')

train_set, test_set = train_test_split(data, test_size=0.1, random_state=42)

KeyboardInterrupt: 

In [ ]:
# extra code
def precision_at_recall(y, y_pred, *, recall, **kwargs):
    """
    Calculate the precision at a given recall level.

    To use this with cross_val_score, you need to use the make_scorer function to
    create a scoring function that can be used with cross_val_score. For example:

        scorer = make_scorer(precision_at_recall, recall=0.9)  # 0.9 is the minimum recall level
        cross_val_score(model, X, y, cv=5, scoring=scorer)

    The default will only work for binary classification problems. You must change the
    average parameter if you want to use for multiclass classification. For example:

        scorer = make_scorer(precision_at_recall, recall=0.9, average="micro")
    """
    return precision_score(y, y_pred, **kwargs) if recall_score(y, y_pred, **kwargs) > recall else 0.0

def recall_at_precision(y, y_pred, *, precision, **kwargs):
    """
    Calculate the recall at a given precision level.

    To use this with cross_val_score, you need to use the make_scorer function to
    create a scoring function that can be used with cross_val_score. For example:

        scorer = make_scorer(recall_at_precision, precision=0.9)
        cross_val_score(model, X, y, cv=5, scoring=scorer)

    The default will only work for binary classification problems. You must change the
    average parameter if you want to use for multiclass classification. For example:

        scorer = make_scorer(recall_at_precision, precision=0.9, average="micro")
    """
    return recall_score(y, y_pred, **kwargs) if precision_score(y, y_pred, **kwargs) > precision else 0.0

In [13]:
# feature lists
weather_features = ['date', 'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation_probability', 'apparent_temperature', 'precipitation',
       'rain', 'showers', 'snowfall', 'snow_depth', 'soil_temperature_0cm',
       'soil_moisture_0_to_1cm', 'vapour_pressure_deficit',
       'et0_fao_evapotranspiration', 'evapotranspiration', 'visibility',
       'cloud_cover_mid', 'cloud_cover_high', 'cloud_cover_low', 'cloud_cover',
       'surface_pressure', 'weather_code', 'pressure_msl', 'wind_speed_10m',
       'wind_speed_80m', 'wind_speed_120m', 'wind_speed_180m',
       'wind_direction_10m', 'wind_direction_80m', 'wind_direction_120m',
       'wind_direction_180m', 'wind_gusts_10m', 'temperature_80m',
       'temperature_120m', 'temperature_180m', 'latitude', 'longitude',
       'IATA']

airplane_features = ["id", "reg", "active", "serial", "hexIaco", "airlineName", "icaoCodeShort", "iataCode", "model", "modelCode", "numSeats", 
"rolloutDate", "firstFlightDate", "deliveryDate", "registrationDate", "typeName", "numEngines", "engineType", "isFreighter", 
"productionLine", "ageYears", "verified", "numRegistrations", "firstRegistrationDate"]

airplane_features_drop = ['id', 'active', 'serial', 'hexIaco', 'icaoCodeShort']

flight_features = ['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
       'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners',
       'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline',
       'Flight_Number_Marketing_Airline',
       'Originally_Scheduled_Code_Share_Airline',
       'DOT_ID_Originally_Scheduled_Code_Share_Airline',
       'IATA_Code_Originally_Scheduled_Code_Share_Airline',
       'Flight_Num_Originally_Scheduled_Code_Share_Airline',
       'Operating_Airline ', 'DOT_ID_Operating_Airline',
       'IATA_Code_Operating_Airline', 'Tail_Number',
       'Flight_Number_Operating_Airline', 'OriginAirportID',
       'OriginAirportSeqID', 'OriginCityMarketID', 'Origin', 'OriginCityName',
       'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac',
       'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'Dest',
       'DestCityName', 'DestState', 'DestStateFips', 'DestStateName',
       'DestWac', 'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes',
       'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut',
       'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrTime', 'ArrDelay',
       'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk',
       'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 
       'AirTime', 'Flights', 'Distance', 'DistanceGroup',
       'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay',
       'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime',
       'DivAirportLandings', 'DivReachedDest', 'DivActualElapsedTime',
       'DivArrDelay', 'DivDistance', 'Div1Airport', 'Div1AirportID',
       'Div1AirportSeqID', 'Div1WheelsOn', 'Div1TotalGTime',
       'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum', 'Div2Airport',
       'Div2AirportID', 'Div2AirportSeqID', 'Div2WheelsOn', 'Div2TotalGTime',
       'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum', 'Div3Airport',
       'Div3AirportID', 'Div3AirportSeqID', 'Div3WheelsOn', 'Div3TotalGTime',
       'Div3LongestGTime', 'Div3WheelsOff', 'Div3TailNum', 'Div4Airport',
       'Div4AirportID', 'Div4AirportSeqID', 'Div4WheelsOn', 'Div4TotalGTime',
       'Div4LongestGTime', 'Div4WheelsOff', 'Div4TailNum', 'Div5Airport',
       'Div5AirportID', 'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime',
       'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum', 'Duplicate',
       'Unnamed: 119']

diverted_features = ['DivAirportLandings','DivReachedDest','DivActualElapsedTime','DivArrDelay','DivDistance','Div1Airport', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn', 'Div1TotalGTime', 'Div1LongestGTime', 'Div1WheelsOff', 'Div1TailNum', 'Div2Airport', 'Div2AirportID', 'Div2AirportSeqID', 'Div2WheelsOn', 'Div2TotalGTime', 'Div2LongestGTime', 'Div2WheelsOff', 'Div2TailNum', 'Div3Airport', 'Div3AirportID', 'Div3AirportSeqID', 'Div3WheelsOn', 'Div3TotalGTime', 'Div3LongestGTime', 'Div3WheelsOff', 'Div3TailNum', 'Div4Airport', 'Div4AirportID', 'Div4AirportSeqID', 'Div4WheelsOn', 'Div4TotalGTime', 'Div4LongestGTime', 'Div4WheelsOff', 'Div4TailNum', 'Div5Airport', 'Div5AirportID', 'Div5AirportSeqID', 'Div5WheelsOn', 'Div5TotalGTime', 'Div5LongestGTime', 'Div5WheelsOff', 'Div5TailNum']

categorical_features = ['weather_code']
numerical_features = weather_features

# drop features
# features that cannot be used because we have to predict 2 weeks ahead of time
too_early_features = ['CRSDepTime','DepTime','DepDelay', 'DepDelayMinutes', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15', 'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode', 'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 'AirTime', 'Flights', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime', 'Duplicate', 'Unnamed: 119', 'Originally_Scheduled_Code_Share_Airline','DOT_ID_Originally_Scheduled_Code_Share_Airline', 'IATA_Code_Originally_Scheduled_Code_Share_Airline', 'Flight_Num_Originally_Scheduled_Code_Share_Airline']
# these features are like to bias the model because they are highly correlated with other features
highly_correlated_weather_features = ['rain', 'temperature_2m', 'precipitation_probability', 'dew_point_2m', 'soil_temperature_0cm', 'soil_moisture_0_to_1cm', 'IATA', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'wind_speed_80m', 'wind_speed_120m', 'wind_speed_180m', 'temperature_80m', 'temperature_120m', 'temperature_180m', 'wind_direction_80m', 'wind_direction_120m', 'wind_direction_180m', 'wind_speed_10m']
highly_correlated_with_other_features  = ['DistanceGroup'] + highly_correlated_weather_features

drop_features = diverted_features + too_early_features + highly_correlated_with_other_features

In [ ]:
# pipeline

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
])

numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer([
    ('cat', categorical_pipeline, categorical_features),
    ('num', numerical_pipeline, numerical_pipeline),
    ('drop', 'drop', drop_features)
])
pickle.dump(preprocessor, open('preprocessor.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
preprocessor


ColumnTransformer(transformers=[('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('encoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['weather_code']),
                                ('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler',
                                                  StandardScaler())])),...
                                  'Div1AirportSeqID', 'Div1WheelsOn',
                                  'Div1TotalGTime', 'Div1LongestGTime',
                                  'Div1WheelsOff', 'Div1TailNum', 'Div2Airport',
                                  'Div2AirportID', 'Div2AirportSeqID',
                                  'Div2WheelsOn', 'Div2TotalGTime',
                                  'Div2LongestGTime', 'Div2WheelsOff',
                                  'Div2TailNum', 'Div3Airport', 'Div3AirportID',
                                  'Div3AirportSeqID', 'Div3WheelsOn',
                                  'Div3TotalGTime', 'Div3LongestGTime',
                                  'Div3WheelsOff', 'Div3TailNum', 'Div4Airport', ...])])

In [ ]:
X, y = None, None

In [ ]:
# models

# LogisticRegression
# - penalty: l2 (default), l1 (for sparse models), elasticnet, or none
# - C: 1 (default), decrease for regularization
# - dual: False (default)
# - solver: different solvers are better for different problems
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', LogisticRegression(max_iter=1000))
    ]),
    param_grid={
        'classifier__penalty': ['l1', 'l2'],
        'classifier__C': [0.01, 0.1, 1, 10, 100],
        'classifier__dual': [False],
        'classifier__solver': ['liblinear', 'saga'],
    },
    scoring='f1',
    cv=5,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('LogisticRegression_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

# - SGDClassifier [basically an online version of LogisticRegression, if online is not needed then you probably don't need this]
# - loss: hinge (default, like LinearSVC) or log (like LogisticRegression)
# - penalty: l2 (default), l1, elasticnet, or none
# - alpha: default 0.0001, increase for regularization
# - l1_ratio: ratio of Lasso vs Ridge if penalty='elasticnet'
# - max_iter/tol: max number of steps to attempt and target tolerance to achieve
# - learning_rate: constant, optimal, invscaling (default), adaptive
# - eta0: initial learning rate, default is 0.01
# - shuffle: default True, shuffle data between iterations
# - early_stopping/validation_fraction/n_iter_no_change: early stopping regularization 
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', SGDClassifier(max_iter=1000))
    ]),
    param_grid={
        'classifier__loss': ['log', 'hinge'],
        'classifier__penalty': ['l1', 'l2'],
        'classifier__alpha': [0.0001, 0.001, 0.01, 0.1],
        'classifier__l1_ratio': [0.15, 0.5, 0.85],
        'classifier__max_iter': [1000, 500],
        'classifier__tol': [0.001, 0.01],
        'classifier__learning_rate': ['optimal', 'constant', 'invscaling'],
        'classifier__eta0': [0.01, 0.1],
        'classifier__shuffle': [True, False],
        'classifier__early_stopping': [True, False],
        'classifier__validation_fraction': [0.1, 0.2],
    },
    scoring='precision_at_recall',
    scoring_params={'recall': 0.25},
    cv=5,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('SGDClassifier_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

# - MLPClassifier (i.e. neural network)
# - hidden_layer_sizes: default is (100,)
# - activation: 'relu' (default), 'identity', 'logistic', 'tanh'
# - alpha: default 0.0001, increase for regularization (always L2)
# - max_iter/tol: max number of steps to attempt and target tolerance to achieve
# - learning_rate_init: initial learning rate, default is 0.001
# - batch_size: sizes of batches
# - shuffle: default True, shuffle data between iterations
# - early_stopping/validation_fraction/n_iter_no_change: early stopping regularization
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', MLPClassifier())
    ]),
    param_grid={
        'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'classifier__activation': ['relu', 'identity', 'logistic', 'tanh'],
        'classifier__alpha': [0.0001, 0.001, 0.01],
        'classifier__max_iter': [1000, 500],
        'classifier__tol': [0.001, 0.01],
        'learning_rate_init': [0.001, 0.01],
        'classifier__batch_size': [10, 20],
        'classifier__solver': ['adam', 'sgd'],
        'early_stopping': [True, False],
        'validation_fraction': [0.1, 0.2],
        'n_iter_no_change': [10, 20],
    },
    scoring='precision_at_recall',
    scoring_params={'recall': 0.25}, 
    cv=5,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('MLPClassifier_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

# - DecisionTreeClassifier
# - criterion: gini (default) or entropy
# - splitter: best (default) or random (faster)
# - max_depth, max_leaf_nodes, min_samples_split, min_samples_leaf, min_impurity_split, 
# - etc: control tree generation, decrease max_* to regularize, increase min_* to regularize
# - presort: setting to True can increase speed for small datasets or restricted depths
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', DecisionTreeClassifier())
    ]),
    param_grid={
        'classifier__criterion': ['gini', 'entropy'],
        'classifier__splitter': ['best', 'random'],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4],
        'classifier__max_leaf_nodes': [None, 10, 20, 30],
        'min_impurity_split': [0.0, 0.1, 0.2],
        'classifier__presort': [True, False]
    },
    scoring='precision_at_recall',
    scoring_params={'recall': 0.25},
    cv=5,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('DecisionTreeClassifier_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

# - GradientBoostingClassifier
# - improved version but requires an external library to be installed and has a bit difference hyperparameters
# - learning_rate: default 0.1, lower to increase regularization, higher to go faster
# - n_estimators: default 100, balance with learning rate, can be fairly high though
# - subsample: default is 1.0, values <1.0 enable stochastic gradient boosting
# - n_iter_no_change/validation_fraction: early stopping regularization
# - Supports most max_* and min_* hyperparameters of DecisionTreeClassifier listed above
# - max_features defaults to sqrt, also supports log2, int (for a count), or float (for a percentage)
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', GradientBoostingClassifier())
    ]),
    param_grid={
        'classifier__learning_rate': [0.01, 0.1, 1],
        'classifier__n_estimators': [50, 100, 200],
        'classifier__subsample': [0.5, 0.75, 1.0],
        'validation_fraction': [0.1, 0.2],
        'n_iter_no_change': [10, 20],
        'classifier__max_depth': [3, 5, 7],
        'classifier__min_samples_split': [2, 5, 10],
        'max_features': ['sqrt', 'log2', 0.5, 0.75, 1.0],
    },
    scoring='precision_at_recall',
    scoring_params={'recall': 0.25},
    cv=5,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('GradientBoostingClassifier_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

# - RandomForestClassifier and ExtraTreesClassifier
# - n_estimators: default 100
# - Supports all hyperparameters of DecisionTreeClassifier listed above except splitter (always best) and presort (always False)
# - max_features defaults to sqrt, also supports log2, int (for a count), or float (for a percentage)
# - max_samples & bootstrap: default all samples with bootstrapping
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', RandomForestClassifier(max_iter=1000))
    ]),
    param_grid={
        'classifier__n_estimators': [50, 100, 200],
        'max_features': ['sqrt', 'log2', 0.5, 0.75, 1.0],
        'max_samples': [0.5, 0.75, 1.0],
        'classifier__bootstrap': [True, False],
    },
    scoring='f1',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('RandomForestClassifier_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

# - ExtraTreesClassifier
# - n_estimators: default 100
# - Supports all hyperparameters of DecisionTreeClassifier listed above except splitter (always best) and presort (always False)
# - max_features defaults to sqrt, also supports log2, int (for a count), or float (for a percentage)
# - max_samples & bootstrap: default all samples with bootstrapping
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', ExtraTreesClassifier(max_iter=1000))
    ]),
    param_grid={
        'classifier__n_estimators': [50, 100, 200],
        'max_features': ['sqrt', 'log2', 0.5, 0.75, 1.0],
        'max_samples': [0.5, 0.75, 1.0],
        'classifier__bootstrap': [True, False],
    },
    scoring='f1',
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('ExtraTreesClassifier_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)

# - KNeighborsClassifier
# - n_neighbors: 5 (default)
# - weights: 'uniform' (default) or 'distance'
grid_search = GridSearchCV(
    estimator=Pipeline([
        ('classifier', KNeighborsClassifier())
    ]),
    param_grid={
        'classifier__n_neighbors': [3, 5, 10],
        'classifier__weights': ['uniform', 'distance'],
        'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
    scoring='precision_at_recall',
    scoring_params={'recall': 0.25},
    cv=5,
    verbose=1,
    n_jobs=-1
)
grid_search.fit(X, y)
print(grid_search.best_params_)
# Save best model
best_model = grid_search.best_estimator_
pickle.dump(best_model, open('KNeighborsClassifier_best_model.pkl', 'wb'), protocol=pickle.HIGHEST_PROTOCOL)